This notebook is intended just for the SVG files in the paper:

"Automatic 3D reconstruction from 2D orthographic drawings"

Assumptions:

1. The structure of the XML files is as follows:
    
        xml info
        svg info
            title
            description
            definitions
            content group
                container group
                    edge container (contains transform and hidden line info)
                        edge projection
                    edge container
                        edge projection
                    .
                    .
                    .
                    edge container
                        edge projection
      
2. All projections are first angle:
    Left view is located to the right of the front view
    Bottom view is located on top of the front view
    etc.
3. The drawing contains exactly three views: Front, Bottom, and Left
4. Transforms specified in the edge container do not include skews or rotations. I am just extracting the translation part.
5. The transform translation information specifies the view, e.g:
    - Front view is
        
        (1485, 1050)
    - Bottom view is
        
        (1485, 700)
    - Left view is
        
        (1800, 1050)
6. All edge elements are given as paths.
7. All line elements are specified with absolute line units "L"
8. All curve elements are specified with absolute cubic Bezier units "C"
9. All curve elements are quarter circles (We'll see about this one)
    - I'm not sure how to calculate the bounding box of a bezier curve.
    - Quarter circles have a bounding box that is easy to calculate

In [ ]:
import re
import numpy as np
from xml.etree import ElementTree as ET
from typing import Tuple, Union

LOOPMAX: int = 1000

In [ ]:
class View:
    def __init__(self, coords: Tuple[float, float], plane: str):
        self.coords: tuple = coords
        self.plane: str = plane
        self.absolute: bool = True
        self.b_box: tuple = ((None, None), (None, None))
        self.elements: list = []
        self.lines: list = []

    def append(self, item) -> None: # item should be a Move, Line, or Curve instance
        self.lines.append(item)

    # Draw is for testing purposes: to visually see if all elements belong to that view.
    def Draw(self, dwg: SVG_Drawing) -> None:
        # file name without extension
        file_name = dwg.file_path.rsplit(".", 1)[0]

        with open(f"{file_name}_{self.plane}.svg", 'x') as file:
            file.write(dwg.header)

            for element in self.elements:
                file.write(ET.tostring(element, encoding='unicode'))

            file.write(dwg.footer)


In [ ]:
class Move:
    def __init__(self, text: str):
        self.text: str = text

        # Split text by commas or spaces (both are valid delimiters)
        move_list: list = re.split(r"[, ]+", self.text)
        # Remove 'M' from first element to make it consist of just numbers, minus sign, and/or decimal points
        move_list[0] = re.sub("M", "", move_list[0])

        self.x: float = float(move_list[0])
        self.y: float = float(move_list[1])
        self.end_coords: tuple = (self.x, self.y)

In [ ]:
class Line:
    def __init__(self, text: str, current_pt: Tuple[float, float], view: View, hidden=False):

        self.text: str = text
        # Split text by commas or spaces (both are valid delimiters)
        line_list: list = re.split(r"[, ]+", self.text)
        # Remove 'M' from first element to make it consist of just numbers, minus sign, and/or decimal points
        line_list[0] = re.sub("L", "", line_list[0])

        self.x1, self.y1 = current_pt
        self.x2, self.y2 = float(line_list[0]), float(line_list[1])
        self.end_coords: tuple = (self.x2, self.y2)

        # Defining projection type
        self.Pt: bool = (self.x1 == self.x2) and (self.y1 == self.y2)
        self.v: bool = (self.x1 == self.x2) and (self.y1 != self.y2)
        self.h: bool = (self.x1 != self.x2) and (self.y1 == self.y2)
        self.I: bool = (self.x1 != self.x2) and (self.y1 != self.y2)

        # Projection type assignment
        if self.Pt:
            self.proj_type = 'Pt'

        if self.v:
            if view.plane == 'XY':
                self.proj_type = 'Py'
            elif view.plane == 'XZ':
                self.proj_type = 'Pz'
            elif view.plane == 'ZY':
                self.proj_type = 'Pz'
            else:
                print('Undefined plane!')

        if self.h:
            if view.plane == 'XY':
                self.proj_type = 'Px'
            if view.plane == 'XZ':
                self.proj_type = 'Px'
            if view.plane == 'ZY':
                self.proj_type = 'Py'

        if self.I:
            self.proj_type = 'I'

        # x1, y1, x2, and y2 form the bounding box
        # Find the lower bound for x and y
        # These evaluate differently if coordinates are relative vs absolute
        self.x_l: float = (self.x1<self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_l: float = (self.y1<self.y2)*self.y1 + (self.y1>=self.y2)*self.y2
        # Find the upper bound for x and y
        self.x_u: float = (self.x1>self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_u: float = (self.y1>self.y2)*self.y1 + (self.y1<=self.y2)*self.y2

        # Construct bounding box
        self.b_box: Tuple[Tuple[float]] = ((self.x1, self.y1), (self.x2, self.y2))

    def transform(self, x: float, y: float) -> tuple:
        return x - self.view_x_l, self.view_y_l - y

    def detransform(self, x: float, y: float) -> tuple:
        return x + self.view_x_l, self.view_y_l + y

    def convert_to_view_rel_coords(self) -> None:
        self.x1, self.y1 = self.transform(self.x1, self.y1)
        self.x2, self.y2 = self.transform(self.x2, self.y2)

        # Need to think about these ones - do x1, x2, y1, and y2 update immediately?
        self.x_l, self.y_l = self.transform(self.x_l, self.y_l)
        self.x_u, self.y_u = self.transform(self)

    def convert_to_abs_coords(self) -> None:
        self.x1, self.y1 = self.detransform(self.x1, self.y1)
        self.x2, self.y2 = self.detransform(self.x2, self.y2)

        # Need to think about these ones - do x1, x2, y1, and y2 update immediately?
        self.x_l, self.y_l = self.detransform(self.x_l, self.y_l)
        self.x_u, self.y_u = self.detransform(self)

In [ ]:
class Curve:
    def __init__(self, text: str, current_pt: Tuple[float, float], view: View, hidden=False):

        self.text: str = text
        self.proj_type: str = 'C'

        # Split text by commas or spaces (both are valid delimiters)
        curve_list: list = re.split(r"[, ]+", self.text)
        # Remove 'M' from first element to make it consist of just numbers, minus sign, and/or decimal points
        curve_list[0] = re.sub("C", "", curve_list[0])

        self.x1, self.y1 = current_pt
        self.cp1 = (float(curve_list[0]), float(curve_list[1]))
        self.cp2 = (float(curve_list[2]), float(curve_list[3]))
        self.x2, self.y2 = (float(curve_list[4]), float(curve_list[5]))
        self.end_coords = (self.x2, self.y2)

        # x1, y1, x2, and y2 form the bounding box
        # Find the lower bound for x and y
        self.x_l: float = (self.x1<self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_l: float = (self.y1<self.y2)*self.y1 + (self.y1>=self.y2)*self.y2
        # Find the upper bound for x and y
        self.x_u: float = (self.x1>self.x2)*self.x1 + (self.x1>=self.x2)*self.x2
        self.y_u: float = (self.y1>self.y2)*self.y1 + (self.y1<=self.y2)*self.y2

        self.b_box: Tuple[float] = (self.x_l, self.y_l, self.x_u, self.y_u)

In [ ]:
class SVG_Drawing:
    def __init__(self, file_path: str, angle_projection: str = 'First'):
        self.file_path: str = file_path
        self.angle_projection: str = angle_projection.lower()
        self.tree = ET.parse(file_path)
        self.root = self.tree.getroot()

        # Extract header and footers
        with open(file_path, 'r') as file:
            content = file.read()
            # rect signifies the start of the elements
            # We want everything before that and including that as our header
            spl = re.split(r"(<rect .+?\n.+?</g>)", content)
            self.header = spl[0] + spl[1]

            self.footer = "  </g>\n </g>\n/</svg>" # I'm assuming this is the footer

        # Handling views
        view_coords: set = set()
        self.line_elements: list = []
        # Looking to extract 'transform' attribute
        # The last two arguments in transform are the x translation and y translation, respectively
        # We skip the first elem because it seems like it's just a rectangle with no size (that's why it's [1:])
        for elem in self.root[3][0][1:]:
            matrix_list = re.split(",", elem.attrib['transform'])
            matrix_list[-1] = re.sub("\)", "", matrix_list[-1])
            view_coords.add((float(matrix_list[-2]), float(matrix_list[-1])))

        # There should be exactly 3 views
        assert len(view_coords) >= 3, "Not enough views"
        assert len(view_coords) == 3, "Too many views"


        # Classify and instantiate Front, Left, and Bottom views
        # Angle projection is assumed to be 1st Angle

        view_coords = sorted(list(view_coords))
        # Left view will be to the right of front view (so x will be greater)
        self.LeftView = View(coords=view_coords[-1], plane='ZY')
        # Bottom view will be on top of the front view (so y will be less, since y is inverted in SVG)
        self.BottomView = View(view_coords[0], plane='XZ')
        # Front view is the only one left
        self.FrontView = View(view_coords[1], plane='XY')
        self.views: list = [self.FrontView, self.LeftView, self.BottomView]

        # Process view characteristics here
        for elem in self.root[3][0][1:]:
            self.sortByView(self.views, elem)


    def sortByLine(self, path_text: str, view: View, hidden=False) -> None:

        # Split segment by alphabetic characters
        # We are leaving out "e" since this appears in cases like "e-15" to indicate exponent
        segments = re.findall(r'[A-Za-z][^A-Za-df-z]*', path_text)

        # Current point is the x1 y1 argument for all paths
        # e.g. if there is the command "L203.2,-203.2 ", the 203.2 and -203.2 are the x2 and y2
        # x1 and y1 are taken as the endpoint of the last segment listed
        current_pt: tuple = (0., 0.)
        for seg in segments:

            if seg[0] == 'M':
                current_pt = (Move(seg).end_coords)
            elif seg[0] == 'L':
                line = Line(text=seg, current_pt=current_pt, view=view, hidden=hidden)
                view.append(line)
                current_pt = line.end_coords
            elif seg[0] == 'C':
                curve = Curve(text=seg, current_pt=current_pt, view=view, hidden=hidden)
                view.append(curve)
                current_pt = curve.end_coords
            else:
                print(f'Non-conforming segment found: {seg}')
                continue


    def sortByView(self, views: list, elem) -> None:

        # Extract element coordinates
        matrix_list = re.split(",", elem.attrib['transform'])
        y_coord = float(re.sub("\)", "", matrix_list[-1]))
        x_coord = float(matrix_list[-2])

        # Extract hidden line information, if any
        hidden = "stroke-dasharray" in elem.attrib

        for view in views:
            if view.coords == (x_coord, y_coord):
                view.elements.append(elem)
                # Path elements are contained in the 'd' attribute
                path_text = elem[0].attrib['d']
                self.sortByLine(path_text=path_text, view=view, hidden=hidden)